## Sun transit time

From Wikipedia, the free encyclopedia (https://en.wikipedia.org/wiki/Sun_transit_time):

"The sun transit time, also referred to as Sun–meridian transit time, is the daily moment when the Sun culminates on the observer's meridian, reaching its highest position in the sky. This solar time is most often used as local noon and therefore will vary with longitude. For example, on April 29, the sun transit time in Boston, Massachusetts is 12:42 p.m., whereas that for New York, New York is 12:53 p.m. This is due to longitudinal distance between the two cities. Sun transit corresponds to solar noon"

### Get current location from IP adress:¶

In [1]:
import requests
import json
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.connect(("8.8.8.8", 80))

full_ip_str = str(s.getsockname()[0])
partial_ip = full_ip_str[:-4]

# URL to send the request to
request_url = 'https://geolocation-db.com/jsonp/' + partial_ip
response = requests.get(request_url)
result = response.content.decode()

# Clean the returned string so it just contains the dictionary data for the IP address
result = result.split("(")[1].strip(")")
# Convert this data into a dictionary
result  = json.loads(result)
my_ip_location = result['latitude'], result['longitude']
#print(result)
#print('------------------')
#print(my_ip_location)
#
## from tuple to float via string
#latstr = str(result['latitude'])
#lat = float(latstr)                                                 
#lonstr = str(result['longitude'])
#lon = float(lonstr)#

## Local noon for current location:

In [2]:
import datetime as dt
from pytz import timezone
from skyfield import almanac
from skyfield.api import N, E, wgs84, load

zone = timezone('Europe/Amsterdam')
now = zone.localize(dt.datetime.now())
midnight = now.replace(hour=0, minute=0, second=0, microsecond=0)
lat, lon = float(result['latitude']), float(result['longitude'])
next_midnight = midnight + dt.timedelta(days=1)

ts = load.timescale()
t0 = ts.from_datetime(midnight)
t1 = ts.from_datetime(next_midnight)
eph = load('de421.bsp')
curr_loc = wgs84.latlon(lat * N, lon * E)

f = almanac.meridian_transits(eph, eph['Sun'], curr_loc)
times, events = almanac.find_discrete(t0, t1, f)

# Select transits instead of antitransits.
times = times[events == 1]

t = times[0]
tstr = str(t.astimezone(zone))[:19]
print('Solar noon:', tstr)

Solar noon: 2022-05-17 13:36:48
